# Imports

In [7]:
from sys import path as syspath
syspath.append("..")

In [15]:
from os import listdir
from os.path import join, abspath
from MISalign.model.mis_file import MisFile,save_mis

# Basic Setup

In [33]:
###
# Enter setup information
###
folder_path=r"..\example\data\set_a" # folder with images
misfile_name="mymis.mis" # name for save file
#
file_ending="jpg" # file extension
file_contains="" # file names must contain - i.e. "sample1-5x", "sample2", "10x", etc.
file_notcontains="" # file names must not contain - i.e. "calibration"
###

In [ ]:
mis_filepath=abspath(join(folder_path,misfile_name))
print(f"Save file filepath: {mis_filepath}")
file_names=[x for x in listdir(folder_path) if x.split(".")[-1]==file_ending]
if file_contains != "":
    file_names=[x for x in file_names if file_contains in x]
if file_notcontains != "":
    file_names=[x for x in file_names if file_notcontains not in x]
file_paths=[abspath(join(folder_path,x)) for x in file_names]
print("\n  ".join(["Image files and filepaths:"]+[f"{fn}: {fp}" for fn,fp in zip(file_names,file_paths)]))

In [ ]:
mis_project=MisFile(image_fps=file_paths)
print(mis_project)
save_mis(mis_filepath,mis_project)

# Advanced Setups

## Image Sets

In [41]:
###
# Enter setup information
###
folder_path=r"..\example\data\set_a" # folder with images
misfile_names=["mymis1.mis","mymis2.mis"] # name for save file
#
file_ending="jpg" # file extension
file_contains="" # file names must contain - i.e. "sample1-5x", "sample2", "10x", etc.
file_notcontains="" # file names must not contain - i.e. "calibration"
###

In [ ]:
mis_filepaths=[abspath(join(folder_path,x)) for x in misfile_names]
mis_dict={mn:mp for mn,mp in zip(misfile_names,mis_filepaths)}
print("\n  ".join(["Save files and filepaths: "]+[f"{mn}: {mp}" for mn,mp in zip(misfile_names,mis_filepaths)]))
file_names=[x for x in listdir(folder_path) if x.split(".")[-1]==file_ending]
if file_contains != "":
    file_names=[x for x in file_names if file_contains in x]
if file_notcontains != "":
    file_names=[x for x in file_names if file_notcontains not in x]
file_paths=[abspath(join(folder_path,x)) for x in file_names]
print("\n  ".join(["Image index, file names, and filepaths:"]+[f"{i} - {fn}: {fp}" for i,(fn,fp) in enumerate(zip(file_names,file_paths))]))

In [ ]:
# user selected image sets using start and end index
image_sets={
    "mymis1.mis":(0,4),
    "mymis2.mis":(5,9),
}
for key,value in image_sets.items():
    print(key,file_names[value[0]:value[1]+1])

In [ ]:
for key,value in image_sets.items():
    mis_fp=mis_dict[key]
    print(mis_fp,"\n  ",file_names[value[0]:value[1]+1])
    mis_project=MisFile(image_fps=file_paths[value[0]:value[1]+1])
    save_mis(mis_fp,mis_project)